In [ ]:
import bcrypt
import csv
import os
from datetime import datetime

# File paths
USERS_FILE = 'users_data.txt'
TASKS_FILE = 'tasks_data.txt'
EXPENSES_FILE = 'expenses_data.csv'

# Utility functions for user authentication
def hash_password(password):
    """Hashes the user's password for storage."""
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt()).decode('utf-8')

def check_password(hashed_password, password):
    """Checks if the entered password matches the hashed password."""
    return bcrypt.checkpw(password.encode('utf-8'), hashed_password.encode('utf-8'))

def is_username_taken(username):
    """Checks if the username already exists in the users file."""
    if os.path.exists(USERS_FILE):
        with open(USERS_FILE, 'r') as file:
            for line in file:
                stored_username, _ = line.strip().split(',', 1)
                if stored_username == username:
                    return True
    return False

def register_user(username, password):
    """Registers a new user, ensuring the username is unique."""
    if is_username_taken(username):
        print("Username is already taken. Try a different one.")
        return False
    
    hashed_password = hash_password(password)
    with open(USERS_FILE, 'a') as file:
        file.write(f"{username},{hashed_password}\n")
    print("Registration successful!")
    return True

def login_user(username, password):
    """Logs in a user if the credentials are valid."""
    if os.path.exists(USERS_FILE):
        with open(USERS_FILE, 'r') as file:
            for line in file:
                stored_username, stored_password = line.strip().split(',', 1)
                if stored_username == username and check_password(stored_password, password):
                    return True
    print("Invalid username or password.")
    return False

# Task Management Functions
def add_task(username, task_description):
    """Adds a new task for the user."""
    if not task_description:
        print("Task description cannot be empty!")
        return
    
    task_id = get_next_task_id(username)
    with open(TASKS_FILE, 'a') as file:
        file.write(f"{username},{task_id},{task_description},Pending\n")
    print(f"Task added with ID {task_id}.")

def get_next_task_id(username):
    """Generates the next task ID for the user."""
    task_ids = []
    if os.path.exists(TASKS_FILE):
        with open(TASKS_FILE, 'r') as file:
            for line in file:
                user, task_id, _, _ = line.strip().split(',', 3)
                if user == username:
                    task_ids.append(int(task_id))
    return max(task_ids, default=0) + 1

def view_tasks(username):
    """Displays all tasks for the logged-in user."""
    if os.path.exists(TASKS_FILE):
        with open(TASKS_FILE, 'r') as file:
            tasks = [line.strip() for line in file if line.startswith(username)]
            if tasks:
                print("Your Tasks:")
                for task in tasks:
                    user, task_id, description, status = task.split(',')
                    print(f"ID: {task_id} | {description} | Status: {status}")
            else:
                print("You have no tasks.")
    else:
        print("No tasks found.")

def mark_task_completed(username, task_id):
    """Marks a task as completed."""
    tasks = []
    task_found = False
    if os.path.exists(TASKS_FILE):
        with open(TASKS_FILE, 'r') as file:
            tasks = file.readlines()
        
        with open(TASKS_FILE, 'w') as file:
            for task in tasks:
                user, tid, description, status = task.strip().split(',', 3)
                if user == username and tid == str(task_id):
                    status = "Completed"
                    task_found = True
                file.write(f"{user},{tid},{description},{status}\n")
    
    if not task_found:
        print("Task not found or already completed.")
    else:
        print(f"Task {task_id} marked as completed.")

def delete_task(username, task_id):
    """Deletes a task for the user."""
    tasks = []
    task_found = False
    if os.path.exists(TASKS_FILE):
        with open(TASKS_FILE, 'r') as file:
            tasks = file.readlines()
        
        with open(TASKS_FILE, 'w') as file:
            for task in tasks:
                user, tid, description, status = task.strip().split(',', 3)
                if user == username and tid == str(task_id):
                    task_found = True
                    continue  # Skip writing this task (delete it)
                file.write(f"{user},{tid},{description},{status}\n")
    
    if task_found:
        print(f"Task {task_id} deleted.")
    else:
        print("Task not found.")

# Expense Management Functions
def add_expense(date, category, amount, description):
    """Adds a new expense."""
    try:
        # Validate date format
        datetime.strptime(date, "%Y-%m-%d")
    except ValueError:
        print("Invalid date format. Please use YYYY-MM-DD.")
        return
    
    if not category or not description:
        print("Category and description cannot be empty.")
        return
    
    try:
        amount = float(amount)
        if amount <= 0:
            print("Expense amount must be greater than zero.")
            return
    except ValueError:
        print("Invalid amount. Please enter a number.")
        return

    with open(EXPENSES_FILE, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([date, category, amount, description])
    print("Expense added.")

def view_expenses():
    """Displays all expenses."""
    if os.path.exists(EXPENSES_FILE):
        with open(EXPENSES_FILE, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                date, category, amount, description = row
                print(f"{date} | {category} | {amount} | {description}")
    else:
        print("No expenses found.")

def track_budget(monthly_budget):
    """Tracks expenses against a monthly budget."""
    if not isinstance(monthly_budget, (int, float)) or monthly_budget <= 0:
        print("Invalid budget amount.")
        return
    
    total_expenses = 0.0
    if os.path.exists(EXPENSES_FILE):
        with open(EXPENSES_FILE, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                total_expenses += float(row[2])  # Amount column

    remaining_budget = monthly_budget - total_expenses
    if remaining_budget < 0:
        print(f"You have exceeded your budget by ${-remaining_budget:.2f}.")
    else:
        print(f"You have ${remaining_budget:.2f} remaining in your budget.")

# Main Program Loop
def main():
    print("=== Welcome to the Task Manager ===")
    username = ""
    while True:
        if not username:
            print("\n1. Register\n2. Login\n3. Exit")
            option = input("Choose an option: ")
            if option == "1":
                user = input("Enter username: ")
                pw = input("Enter password: ")
                if register_user(user, pw):
                    username = user
            elif option == "2":
                user = input("Enter username: ")
                pw = input("Enter password: ")
                if login_user(user, pw):
                    username = user
            elif option == "3":
                break
            else:
                print("Invalid option.")
        else:
            print(f"\nWelcome, {username}!")
            print("1. Add Task\n2. View Tasks\n3. Mark Task as Completed\n4. Delete Task\n5. Add Expense\n6. View Expenses\n7. Track Budget\n8. Logout")
            option = input("Choose an option: ")
            if option == "1":
                task_desc = input("Enter task description: ")
                add_task(username, task_desc)
            elif option == "2":
                view_tasks(username)
            elif option == "3":
                task_id = int(input("Enter task ID to mark as completed: "))
                mark_task_completed(username, task_id)
            elif option == "4":
                task_id = int(input("Enter task ID to delete: "))
                delete_task(username, task_id)
            elif option == "5":
                date = input("Enter expense date (YYYY-MM-DD): ")
                category = input("Enter expense category: ")
                amount = input("Enter expense amount: ")
                description = input("Enter expense description: ")
                add_expense(date, category, amount, description)
            elif option == "6":
                view_expenses()
            elif option == "7":
                budget = float(input("Enter your monthly budget: "))
                track_budget(budget)
            elif option == "8":
                print("Logging out...")
                username = ""
            else:
                print("Invalid option.")

if __name__ == "__main__":
    main()


=== Welcome to the Task Manager ===

1. Register
2. Login
3. Exit
Choose an option: 1
Enter username: Manav
Enter password: Manav
Registration successful!

Welcome, Manav!
1. Add Task
2. View Tasks
3. Mark Task as Completed
4. Delete Task
5. Add Expense
6. View Expenses
7. Track Budget
8. Logout
Choose an option: 1
Enter task description: Complete Station Audit
Task added with ID 1.

Welcome, Manav!
1. Add Task
2. View Tasks
3. Mark Task as Completed
4. Delete Task
5. Add Expense
6. View Expenses
7. Track Budget
8. Logout
Choose an option: 1
Enter task description: Complete OCC room audit
Task added with ID 2.

Welcome, Manav!
1. Add Task
2. View Tasks
3. Mark Task as Completed
4. Delete Task
5. Add Expense
6. View Expenses
7. Track Budget
8. Logout
Choose an option: 2
Your Tasks:
ID: 1 | Complete Station Audit | Status: Pending
ID: 2 | Complete OCC room audit | Status: Pending

Welcome, Manav!
1. Add Task
2. View Tasks
3. Mark Task as Completed
4. Delete Task
5. Add Expense
6. View Ex